In [6]:
from chemicalbd.bond_disconnector import C_S_disconnection
from rdkit import Chem

mol = Chem.MolFromSmiles('CSCC')
reactant_list = C_S_disconnection(mol)[1]
    assert Chem.MolToSmiles(reactant_list[0]) == Chem.MolToSmiles(Chem.MolFromSmiles('CI'))

In [5]:
 assert Chem.MolToSmiles(reactant_list[0]) == Chem.MolToSmiles(Chem.MolFromSmiles('CI'))

NameError: name 'reactant_list' is not defined